In [ ]:
%matplotlib inline
from matplotlib import style
style.use('ggplot')
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
from netCDF4 import Dataset, date2index, num2date
from palettable import colorbrewer
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import requests
def listFD(url, ext=''):
    page = requests.get(url).text
    #print(page)
    soup = BeautifulSoup(page, 'html.parser')
    return [url + node.get('href') for node in soup.find_all('a') if (node.get('href').endswith(ext) and node.get('href').startswith('2'))]


In [ ]:
from os.path import exists

filename_viirs_proc = 'F:/data/cruise_data/saildrone/baja-2018/viirs_l2p_files.npy'
filename_viirs_sst = 'F:/data/cruise_data/saildrone/baja-2018/viirs_sst_l2p_data.npy'
filename_png = 'F:/data/cruise_data/saildrone/baja-2018/recent_viirs_sst_l2p_.png'


In [ ]:

for incr_day in range(0,1):
    d = datetime.today() + timedelta(days=incr_day)
    day_of_year = d.timetuple().tm_yday

    url = 'https://opendap.jpl.nasa.gov/opendap/OceanTemperature/ghrsst/data/GDS2/L2P/VIIRS_NPP/OSPO/v2.41/' \
            + str(d.year) + '/' + str(day_of_year).zfill(3) + '/'
    ext = 'nc'

    filenames=listFD(url, ext)
    ilen=len(filenames)
    for ic in range(1,ilen):
        file = filenames[ic]
        print(file)
        #save which files have been processed
        idyj=int(file[101:104])
        ihr=int(file[113:115])
        imin=int(file[115:117])
        print(idyj,ihr,imin)

        nc = Dataset(file)
        lat2 = nc.variables['lat'][:] #[2500:3250]
        nc.close()
        maxlat = np.ma.max(lat2)
        minlat = np.ma.min(lat2)

        ilat_check=0
        if minlat<40.0 and maxlat>40.0:
            ilat_check=1
        if minlat>25.0 and maxlat<40.0:
            ilat_check=1
        if minlat<25.0 and maxlat>25.0:
            ilat_check=1
        if ilat_check==0:
            continue  #no data at right latitude
            
        nc = Dataset(file)
        lon2 = nc.variables['lon'][:] #[2500:3250]
        nc.close()
        
        minlon = np.ma.min(lon2)
        print(maxlat,minlat)
        if minlon>0:
            continue

        minlon = np.ma.min(lon2[lon2<0])
        maxlon = np.ma.max(lon2[lon2<0])
        
        ilon_check=0
        if minlon<-130 and maxlon>-130.0:
            ilon_check=1
        if minlon>-130 and maxlon<-110.0:
            ilon_check=1
        if minlon<-110 and maxlon>-110.0:
            ilon_check=1
        if ilon_check==1 and ilat_check==1:
            nc = Dataset(file)
            sst = nc.variables['sea_surface_temperature'][:] #:,2500:3250,2500:3500]
            nc.close()
            print(sst.shape,lat2.shape,lon2.shape)
            break

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
#print(lon2[lon2>0].count())
#print(lon2[lon2<0].count())
print(sst.shape,lon2.shape,lat2.shape)
#if minlon<-110.0 and minlon
#25.03 39.99 -129.97 -110.01
#plt.scatter(lon,lat)
#plt.show()


print('drawing map....') # "Drawing map... ",
m = Basemap(projection='cyl', llcrnrlat=minlat, urcrnrlat=maxlat,\
               llcrnrlon=minlon, urcrnrlon=maxlon, resolution='l')
m.drawcoastlines()
m.drawmapboundary(fill_color='coral') 
#m.scatter(xlist, ylist, 20, c=confidence, cmap=p.cm.hot, marker='o', edgecolors='none', zorder=10)
#m.imshow(sst)
print('done') # "done !"




In [ ]:
#import numpy as np
#from pyresample import image, geometry
#area_def = geometry.AreaDefinition('areaD', 'Europe (3km, HRV, VTC)', 'areaD',
#...                                {'a': '6378144.0', 'b': '6356759.0',
#...                                 'lat_0': '50.00', 'lat_ts': '50.00',
#...                                 'lon_0': '8.00', 'proj': 'stere'},
#...                                800, 800,
#...                                [-1370912.72, -909968.64,
#...                                 1029087.28, 1490031.36])
#swath_def = geometry.SwathDefinition(lons=lon2, lats=lat2)
#swath_con = image.ImageContainerNearest(sst, swath_def, radius_of_influence=5000)
#area_con = swath_con.resample(area_def)
#result = area_con.image_data

In [ ]:
from palettable import colorbrewer
from copy import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.mlab as mlab
palette = copy(plt.cm.jet)
palette.set_over('r', 1.0)
palette.set_under('g', 1.0)
palette.set_bad(alpha = 0.0)
#fig = plt.figure(figsize=(6, 5.4))
# plot using 'continuous' color map
print(sst.shape)
fig, ax = plt.subplots()
im = ax.imshow(sst[0,:,:].T-273.15, interpolation='bilinear',cmap=palette,norm=colors.Normalize(vmin=10, vmax=17.0),aspect='auto',origin='lower')
# We want to show all ticks...
ax.set_xticks(lon2)
ax.set_yticks(lat2)
# ... and label them with the respective list entries
#ax.set_xticklabels(farmers)


#extent=[x0, x1, y0, y1])
#fig.set_title('SST')
#cbar = fig.colorbar(im, extend='both', shrink=0.9, ax=im)
#cbar.set_label('uniform')
#for ticklabel in ax1.xaxis.get_ticklabels():
#   ticklabel.set_visible(False)
plt.show()

In [ ]:
        if inew_data==1:
            sst_new = ma.copy(sst)
            inew_data = 0
        if cnt<5:
            continue
        print(ic,cnt,file)
        mask = sst_new < -10
        sst_new[mask] = sst[mask]
     
if itotal_proc>0:
    lat, lon = np.meshgrid(lat, lon)

    mask = (sst_new < -10)
    sst_new[mask]=-9999.
    sst_new2 = sst_new[0,:,:]
    sstx = ma.masked_values (sst_new2, -9999.)
    print(lon.shape,lat.shape,sstx.shape)
    sst3x = np.flip(sstx, 1)
    sst4x = np.flip(sst3x, 0)
    sst5x = ma.swapaxes(sst4x,0,1)
    print(lon.shape,lat.shape,sst5x.shape)

    pixels = 1024 * 10
    cmap = colorbrewer.get_map('Spectral', 'diverging', 11, reverse=True).mpl_colormap

    fig, ax = gearth_fig(llcrnrlon=lon.min(),
                                     llcrnrlat=lat.min(),
                                     urcrnrlon=lon.max(),
                                     urcrnrlat=lat.max(),
                                     pixels=pixels)
    cs = ax.pcolormesh(lon, lat, sst5x - 273.15, cmap=cmap, vmin = 10, vmax = 17)
    ax.set_axis_off()
    fig.savefig(filename_png, transparent=False, format='png')
    print('done')        
    print(lat[1,1],lat[1,-1],lon[1,1],lon[-1,1])
    make_kml(llcrnrlon=lon[1,1], llcrnrlat=lat[1,1],
             urcrnrlon=lon[-1,1], urcrnrlat=lat[1,-1],
             figs= [filename_png], colorbar='legend.png',
             kmzfile=filename_kmz, name='VIIRS Sea Surface Temperature')
    np.save(filename_viirs_proc, proc_save)
    sst_new.dump(filename_viirs_sst)

fig = plt.figure(figsize=(1.0, 4.0), facecolor=None, frameon=False)
ax = fig.add_axes([0.0, 0.05, 0.2, 0.9])
cb = fig.colorbar(cs, cax=ax)
cb.set_label('SST [C]', rotation=-90, color='k', labelpad=20)
fig.savefig('legend.png', transparent=False, format='png')  # Change transparent to True if your colorbar is not on space :)


In [ ]:
#now just do todays data
from os.path import exists

filename_png = 'F:/data/cruise_data/saildrone/baja-2018/today_viirs_sst.png'
filename_kmz = 'F:/data/cruise_data/saildrone/baja-2018/todat_viirs_sst.kmz'
inew_data=1

for incr_day in range(-1,1):
    d = datetime.today() + timedelta(days=incr_day)
    day_of_year = d.timetuple().tm_yday

    url = 'https://opendap.jpl.nasa.gov/opendap/OceanTemperature/ghrsst/data/GDS2/L3U/VIIRS_NPP/OSPO/v2.41/' \
            + str(d.year) + '/' + str(day_of_year).zfill(3) + '/'
    ext = 'nc'

    filenames=listFD(url, ext)
    ilen=len(filenames)
    for ic in range(1,ilen):
        file = filenames[ic]
        
        #save which files have been processed
        idyj=int(file[101:104])
        ihr=int(file[113:115])
        imin=int(file[115:117])
            
        nc = Dataset(file)
        sst = nc.variables['sea_surface_temperature'][:,2500:3250,2500:3500]
        sst = np.flip(sst, 2)
        cnt = ma.count(sst)
        lat = nc.variables['lat'][2500:3250]
        lat = np.flip(lat, 0)
        lon = nc.variables['lon'][2500:3500]
        nc.close()
        if inew_data==1:
            sst_new = ma.copy(sst)
            inew_data = 0
        if cnt<5:
            continue
        print(ic,cnt,file)
        mask = sst_new < -10
        sst_new[mask] = sst[mask]
        
    lat, lon = np.meshgrid(lat, lon)
    mask = (sst_new < -10)
    sst_new[mask]=-9999.
    sst_new2 = sst_new[0,:,:]
    sstx = ma.masked_values (sst_new2, -9999.)
    print(lon.shape,lat.shape,sstx.shape)
    sst3x = np.flip(sstx, 1)
    sst4x = np.flip(sst3x, 0)
    sst5x = ma.swapaxes(sst4x,0,1)
    print(lon.shape,lat.shape,sst5x.shape)

    pixels = 1024 * 10
    cmap = colorbrewer.get_map('Spectral', 'diverging', 11, reverse=True).mpl_colormap

    fig, ax = gearth_fig(llcrnrlon=lon.min(),
                                     llcrnrlat=lat.min(),
                                     urcrnrlon=lon.max(),
                                     urcrnrlat=lat.max(),
                                     pixels=pixels)
    cs = ax.pcolormesh(lon, lat, sst5x - 273.15, cmap=cmap, vmin = 10, vmax = 17)
    ax.set_axis_off()
    fig.savefig(filename_png, transparent=False, format='png')
    print('done')        
    print(lat[1,1],lat[1,-1],lon[1,1],lon[-1,1])
    make_kml(llcrnrlon=lon[1,1], llcrnrlat=lat[1,1],
             urcrnrlon=lon[-1,1], urcrnrlat=lat[1,-1],
             figs= [filename_png], colorbar='legend.png',
             kmzfile=filename_kmz, name='VIIRS Sea Surface Temperature')

fig = plt.figure(figsize=(1.0, 4.0), facecolor=None, frameon=False)
ax = fig.add_axes([0.0, 0.05, 0.2, 0.9])
cb = fig.colorbar(cs, cax=ax)
cb.set_label('SST [C]', rotation=-90, color='k', labelpad=20)
fig.savefig('legend.png', transparent=False, format='png')  # Change transparent to True if your colorbar is not on space :)


In [ ]:
pixels = 1024 * 10
cmap = colorbrewer.get_map('Spectral', 'diverging', 11, reverse=True).mpl_colormap
fig, ax = gearth_fig(llcrnrlon=lon.min(),
                                     llcrnrlat=lat.min(),
                                     urcrnrlon=lon.max(),
                                     urcrnrlat=lat.max(),
                                     pixels=pixels)
cs = ax.pcolormesh(lon, lat, sst5x - 273.15, cmap=cmap, vmin = 10, vmax = 17)
ax.set_axis_off()
fig.show()
 

In [ ]:
from copy import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.mlab as mlab
print(sst5x.shape)
print(sst4x.shape)
print(sst3x.shape)
palette = copy(plt.cm.jet)
palette.set_over('r', 1.0)
palette.set_under('g', 1.0)
palette.set_bad(alpha = 0.0)
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(6, 5.4))
# plot using 'continuous' color map
im = ax1.imshow(sst5x - 273.15, interpolation='bilinear',
#im = ax1.imshow(sst4x[540:560,280:289] - 273.15, interpolation='bilinear',
                cmap=palette,
                norm=colors.Normalize(vmin=10.0, vmax=17.0),
                aspect='auto',
                origin='lower')
                #extent=[x0, x1, y0, y1])
ax1.set_title('sst')

plt.show()
print(lon[283,546],lat[283,546])
